In [ ]:
# REFERENCES

# [1] https://www.tensorflow.org/federated/api_docs/python/tff/federated_computation
# [2] TensorFlow Federated Tutorial Session, Google Tech Talk, Youtube, https://www.youtube.com/watch?v=JBNas6Yd30A

# REQUIREMENTS

# Might need to first install CMake
# $ pip install CMake

# tensorflow-federated: A Python open-source framework for machine learning and other computations on decentralized data.
# $ pip install --upgrade tensorflow-federated

# nest-asyncio: A Python module that patches asyncio to allow nested use of asyncio.run and loop.run_until_complete.
# $ pip install --upgrade nest-asyncio

In [1]:
# Local networking stuff. Cant run on Google Colab due to security reasons.
# Allowing the nested use of asyncio.run and Loop.run_until_complete.
import nest_asyncio
nest_asyncio.apply()

In [2]:
# TensorBoard provides the visualization and tooling needed for machine learning experimentation
%load_ext tensorboard

In [4]:
import collections

import numpy as np
import tensorflow as tf
import tensorflow_federated as tff

np.random.seed(0)

# A Federated Computation (or FC) is a computation where the data stays on clients' machines [2]
# This function simulates a federated computation given any python computation [1]
tff.federated_computation(lambda: 1+1)()

# Federated Computations are made of three steps:
#       1. Federated Broadcast: Publicly broadcast Global Model to clients.
#       2. Federated Map: Privately train local parameters using private data and Global Model on the client-side.
#       3. Federated Mean: Aggregate clients' locally trained parameters.

2

In [ ]:
# Federated data is split amongst the clients and privately used in client-side Federated Mapping
# To make things easy for demos, Tensorflow made MNIST into a federated data set by 
# keying the data by the original writer of the digits.
emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data()

In [ ]:
# Client IDs
print(len(emnist_train.client_ids))
print(emnist_train.client_ids)

In [ ]:
client_dataset = emnist_train.create_tf_dataset_for_client(emnist_train.client_ids[533])

example_element = next(iter(example_dataset))

example_element

In [ ]:
from matplotlib import pyplot as plt
plt.imshow(example_element['pixels'].numpy(), cmap='gray', aspect='equal')
plt.grid(False)
_ = plt.show()